### This notebook describes the relevant functions and steps we followed in our project

In [1]:
# Necessary imports 

import numpy as np
import pytket as tk
from pytket import passes as tkp
from pytket import circuit as tkc
from pytket.circuit.display import render_circuit_jupyter as print_circ
from pytket.qasm import circuit_from_qasm_str, circuit_to_qasm, circuit_from_qasm
import zipfile
import shutil
import os


### Ensuring Clifford + T gateset
Before starting with the steps for obtaining the normal form, we present three functions designed to ensure that quantum circuits <br> 
are written into the Clifford+T gateset, and also ensure that the gates are consistently ordered across all circuit files. Then we save the circuit as new QASM files

### Functions

### 1. `cliff_t_rebase() -> tkp.BasePass`: Provided by our mentors
- **Purpose**: Converts single-qubit gates in a quantum circuit to a Clifford+T gateset, which includes H, Z, S, T, and potentially X and V gates.
- **Returns**: A `tkp.BasePass` object configured to enforce the specified gateset transformation.
- **Details**: The function raises an error if the decomposition is not exact to ensure precision in transformations. Notably, this gateset is only approximately universal.

### 2. `qasm_ordering(qasm_file, output_file)`
- **Purpose**: Converts a QASM file to use a Clifford+T gateset and writes the modified circuit back to a file.
- **Parameters**:
  - `qasm_file` (str): Path to the input QASM file.
  - `output_file` (str): Path to save the transformed QASM file.
- **Details**: Reads a QASM file, applies the Clifford+T transformation, and outputs the modified circuit.

### 3. `read_qasm_file(file_path) -> str`
- **Purpose**: Reads and returns the content of a QASM file as a string.
- **Parameters**:
  - `file_path` (str): Path to the QASM file.
- **Returns**: The contents of the QASM file as a string.

## Example Usage
```python
# Apply Clifford+T rebase and export the modified circuit to a new QASM file
qasm_input_path = 'path/to/original.qasm'
qasm_output_path = 'path/to/modified.qasm'
qasm_ordering(qasm_input_path, qasm_output_path)


In [2]:
def cliff_t_rebase() -> tkp.BasePass:
    """Pass to convert single-qubit gates to the Clifford+T gateset.

    Concretely, single-qubit gates will be one of H, Z, S or T. Could also be
    updraded to handle X and V. Have a try!

    pyTKET won't do this for you automatically, because this is not a universal
    gateset (only approximately universal).

    For our purpose, if the decomposition is not exact, we raise an error
    """
    cx_replacement = tk.Circuit(2).CX(0, 1)
    def tk1_replacement(a, b, c, eps=1e-6):
        # make sure the phases are in the range [0, 4)
        a, b, c = a % 4, b % 4, c % 4
        ret = tk.Circuit(1)
        def add_phase(f: float):
            while f > eps:
                if f + eps > 1.:
                    ret.Z(0)
                    f -= 1
                elif f + eps > 0.5:
                    ret.S(0)
                    f -= 0.5
                elif f + eps > 0.25:
                    ret.T(0)
                    f -= 0.25
                else:
                    break
            return f
        rest_c = add_phase(c)
        ret.H(0)
        rest_b = add_phase(b)
        ret.H(0)
        rest_a = add_phase(a)

        if abs(rest_a) > eps or abs(rest_b) > eps or abs(rest_c) > eps:
            raise ValueError("Phases are not multiples of pi/4")
        return ret

    return tkp.RebaseCustom(
        {tk.OpType.CX, tk.OpType.H, tk.OpType.S, tk.OpType.T},
        cx_replacement=cx_replacement,
        tk1_replacement=tk1_replacement
    )

def qasm_ordering(qasm_file, output_file):
    qasm_string = read_qasm_file(qasm_file) # Call function to read .qasm file and save as a string
    qc_from_qasm = circuit_from_qasm_str(qasm_string, maxwidth=1000) # Turn str into circuit
    cliff_opt = tkp.SequencePass([cliff_t_rebase()])
    cliff_opt.apply(qc_from_qasm)
    ordered_qasm_file = circuit_to_qasm(qc_from_qasm, output_file)

def read_qasm_file(file_path):
    """
    Reads the content of a QASM file into a string
    """
    with open(file_path, 'r') as file:
        qasm_str = file.read()
    return qasm_str




## Gadgetizing Hadamards
Description of three functions used to gadgetize hadamard gates: we modify quantum circuits by replacing Hadamard gates with Heyfron-Campbell gadgetization.

## Functions

### 1. `adding_registers(reading_file, modified_file)`
- **Purpose**: The function performs the following operations:
  - Counts each occurrence of a Hadamard gate and notes the qubit involved, and keeps note of their position
  - It appends appends quantum and classical registers needed for gadgetizing the hadamards
- **Parameters**:
  - `reading_file` (str): Input QASM file path.
  - `modified_file` (str): Output QASM file path.
- **Returns**: List of qubit indices where Hadamard gates were detected.

### 2. `hadamard_string(qbit_number, ancilla_number)`
- **Purpose**: Creates a QASM string that substitutes a Hadamard gate with the Heyfron-Campbell gadgetization using an ancilla qubit.
- **Parameters**:
  - `qbit_number` (int): Qubit index of the Hadamard gate.
  - `ancilla_number` (int): Ancilla qubit index.
- **Returns**: String with QASM commands.

### 3. `add_h_blocks(old_file, modified_file, h_qbit_list)`
- **Purpose**: Replaces Hadamard gates with the gate sequence defined in the Heyfron-Campbell paper in a QASM file.
- **Parameters**:
  - `old_file` (str): Original QASM file path.
  - `modified_file` (str): Modified QASM file path.
  - `h_qbit_list` (List[int]): Indices of qubits with Hadamard gates.
- **Returns**: None (outputs directly to a file).

## Example Usage
```python
original_qasm_path = 'path/to/original.qasm'
modified_qasm_path = 'path/to/modified.qasm'
h_qbit_list = adding_registers(original_qasm_path, temp_qasm_path)
add_h_blocks(temp_qasm_path, modified_qasm_path, h_qbit_list)


In [3]:
def adding_registers(reading_file, modified_file):
    '''
    Counts the number of H gates h_count, adds a flag at
    each hadamard, and creates a quantum register and classical
    register of size h_count.
    '''
    h_count = 0
    h_qbit_list = []
    with open(reading_file, 'r') as file, open('temp.qasm', 'w') as new_file:
        for line in file:
            if line.strip().startswith('h q['):
                h_count += 1
                start_idx = line.find('[')
                end_idx = line.find(']')
                h_qbit_list.append(int(line[start_idx+1:end_idx]))
                new_file.write('//spotted\n')
            new_file.write(line)
    file.close()
    new_file.close()

    with open('temp.qasm', 'r') as file, open(modified_file, 'w') as new_file:
        for line in file:
            if line.strip().startswith('qreg'):
                new_file.write(line)
                new_file.write(f'qreg a[{h_count}];\n')
                new_file.write(f'creg ma[{h_count}];\n')
            else:
                new_file.write(line)


    return h_qbit_list

def hadamard_string(qbit_number, ancilla_number):
    '''
    Creates the two qubit blocks that
    substitute the hadamard gates
    '''

    return          f'h a[{ancilla_number}];\n' \
                    +   f's q[{qbit_number}];\n' \
                    +   f's a[{ancilla_number}];\n' \
                    +   f'cx q[{qbit_number}], a[{ancilla_number}];\n' \
                    +   f'sdg a[{ancilla_number}];\n' \
                    +   f'cx a[{ancilla_number}], q[{qbit_number}];\n' \
                    +   f'cx q[{qbit_number}], a[{ancilla_number}];\n' \
                    +   f'h a[{ancilla_number}];\n' \
                    +   f'measure a[{ancilla_number}] -> ma[{ancilla_number}];\n' \
                    +   f'if (ma[{ancilla_number}] == 1) x q[{qbit_number}];\n'

def add_h_blocks(old_file, modified_file, h_qbit_list):
    '''
    Substitutes the hadamard gates
    by the two qubit blocks defined in the paper
    '''
    flag = False
    ancilla_idx = 0

    with open(old_file, 'r') as file, open(modified_file, 'w') as new_file:
        for line in file:
            if flag:
                flag = False
                continue
            else:
                pass

            if line.strip().startswith('//spotted'):
                flag = True
                new_file.write(line)
                new_file.write(hadamard_string(h_qbit_list[ancilla_idx], len(h_qbit_list) - 1 - ancilla_idx))
                ancilla_idx += 1
            else:
                new_file.write(line)

#### Gadgetize QASM Files

This function `gadgetize_qasm_files` processes multiple QASM files to gadgetize them. It takes three parameters:

- `source_dir`: Directory containing input QASM files.
- `target_dir`: Directory to store modified QASM files with added gadgets.
- `log_dir`: Directory to store the lists of indices for the positions of the hadamards


In [ ]:
def gadgetize_qasm_files(source_dir, target_dir, log_dir):
    os.makedirs(target_dir, exist_ok=True)
    os.makedirs(log_dir, exist_ok=True)  # Ensure the log directory exists
    for filename in os.listdir(source_dir):
        if filename.endswith('.qasm'):
            reading_file = os.path.join(source_dir, filename)
            temp_file = os.path.join(target_dir, 'temp_' + filename)
            modified_file = os.path.join(target_dir, filename)
            h_qubit_list = adding_registers(reading_file, temp_file)
            add_h_blocks(temp_file, modified_file, h_qubit_list)
            os.remove(temp_file)

            # Create a log file for each qasm file with the same base name
            log_filename = os.path.join(log_dir, filename.replace('.qasm', '.txt'))
            with open(log_filename, 'w') as log_file:
                log_file.write(f"{filename}: {h_qubit_list}\n")



### Function `return_Uf`

**Purpose**: Processes and modifies a QASM file based on the flags added previously, writing U_f segments to new files and the residual part <br>
that can be written as CNOT+T as another file

#### Parameters
- `filename` (str): Path to the original QASM file.
- `filename_2` (str): Path for the final residual CNOT+T part
- `flag` (str): Substring to search for within the file, marking lines for special processing.

#### Behavior
- **File Creation:** Generates new files (`U_{counter}.qasm`) for each found `flag`, containing parts of the original file after specified deletions.
- **File Modification:** Modifies the original file content by replacing flagged lines and deleting subsequent lines, writing the result to `filename_2`.


In [ ]:
def return_Uf(filename, filename_2, folder_U, flag):
    with open(filename, 'r') as f:
        lines = f.readlines()
    f.close()
    counter = 0

    while True:
      for i in range(len(lines)-1, -1, -1):
          if flag in lines[i]:
              lines[i] = "//flag\n"
              print(i)
              break

      if i == 0:
          break

      else:
          del lines[i+8:i+11]
          with open(folder_U+f"/U_{counter}.qasm", 'w') as f:
              f.writelines(lines[:4])
              if counter:
                f.writelines([f'qreg a[{counter}];\n'])
              else:
                f.write('\n')
              f.writelines(lines[i+8:])
          counter += 1
          f.close()

    with open(filename_2, 'w') as f:
        f.writelines(lines)
    f.close()


### Generate U-Dagger QASMs

The function `gen_u_dag_qasms` generates the QASM files for the inverse (dagger) of the U_i operators obtained previously

### Usage

```python
gen_u_dag_qasms(h_qbit_list)


In [ ]:
def gen_u_dag_qasms(h_qbit_list):
    h_num = len(h_qbit_list)
    for i in range(h_num):
        u = circuit_from_qasm(f'U_{i}.qasm')
        u_dag = u.inverse()
        circuit_to_qasm(u_dag,'U_{i}_dag.qasm')